In [1]:
import os
import sys

PROJECT_MARKERS = ("src", "data", "prompts", "results")

def find_project_root(start_path):
    current = os.path.abspath(start_path)

    while True:
        if all(os.path.isdir(os.path.join(current, m)) for m in PROJECT_MARKERS):
            return current

        parent = os.path.dirname(current)
        if parent == current:
            raise RuntimeError("Project root not found")

        current = parent


# ---- execution directory (cwd) ----
cwd = os.getcwd()

# ---- safe starting point ----
try:
    start_path = os.path.dirname(os.path.abspath(__file__))
except NameError:
    start_path = cwd


# ---- resolve canonical paths ----
project_root = find_project_root(start_path)

# ✅ THIS IS THE IMPORTANT PART
if project_root not in sys.path:
    sys.path.insert(0, project_root)

src_root     = os.path.join(project_root, "src", "daniel", "gemini")
data_root    = os.path.join(project_root, "data", "MAMS-ACSA", "raw", "data_jsonl", "annotated")
schemas_root = os.path.join(project_root, "data", "MAMS-ACSA", "raw", "data_jsonl", "schema")
prompts_root = os.path.join(project_root, "prompts", "daniel", "llama")
utils_root   = os.path.join(project_root, "utils")
results_root = os.path.join(project_root, "results", "daniel")

print(
    f"📂 cwd          : {cwd}\n"
    f"📂 Project root : {project_root}\n"
    f"📂 Source root  : {src_root}\n"
    f"📂 Data root    : {data_root}\n"
    f"📂 Prompts root : {prompts_root}\n"
    f"📂 Utils root   : {utils_root}\n"
    f"📂 Results root : {results_root}"
)

📂 cwd          : /Users/hd/Desktop/RCS-Emotion-Prediction-2025/src/daniel/metrics
📂 Project root : /Users/hd/Desktop/RCS-Emotion-Prediction-2025
📂 Source root  : /Users/hd/Desktop/RCS-Emotion-Prediction-2025/src/daniel/gemini
📂 Data root    : /Users/hd/Desktop/RCS-Emotion-Prediction-2025/data/MAMS-ACSA/raw/data_jsonl/annotated
📂 Prompts root : /Users/hd/Desktop/RCS-Emotion-Prediction-2025/prompts/daniel/llama
📂 Utils root   : /Users/hd/Desktop/RCS-Emotion-Prediction-2025/utils
📂 Results root : /Users/hd/Desktop/RCS-Emotion-Prediction-2025/results/daniel


In [2]:
# ============================================================
# EMOTION-ONLY EVALUATION WITH MISMATCH LOGGING
# ============================================================

import json
import os
import pandas as pd
from sklearn.metrics import f1_score, classification_report

# -----------------------------
# Paths
# -----------------------------

GOLD_PATH = os.path.join(data_root, "02_iteration_cleaned_300.jsonl")
#PRED_PATH = os.path.join(results_root, "llama", "llama_output.jsonl")
#PRED_PATH = os.path.join(results_root, "claude", "claude_output.jsonl")
PRED_PATH = os.path.join(results_root, "gpt", "gpt_output.jsonl")

# ---- derive input identifier ----
input_name = os.path.splitext(os.path.basename(PRED_PATH))[0] # prints model

# -----------------------------
# Load JSONL
# -----------------------------
def load_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        return [json.loads(line) for line in f]

gold = load_jsonl(GOLD_PATH)
pred = load_jsonl(PRED_PATH)

# Map gold/pred by input text
gold_map = {row["input"]: row["output"] for row in gold}
pred_map = {row["input"]: row["output"] for row in pred}

print(f"Loaded gold rows: {len(gold_map)}")
print(f"Loaded pred rows: {len(pred_map)}")

# -----------------------------
# EMOTION-ONLY ALIGNMENT
# -----------------------------
gold_emotions = []
pred_emotions = []
mismatches = []

for text, gold_list in gold_map.items():

    pred_list = pred_map.get(text, [])

    # Emotion-only invariant:
    # - same number of triples
    if len(pred_list) != len(gold_list):
        mismatches.append({
            "input": text,
            "gold_count": len(gold_list),
            "pred_count": len(pred_list),
            "gold_output": gold_list,
            "pred_output": pred_list
        })
        continue

    for gold_triple, pred_triple in zip(gold_list, pred_list):
        gold_emotions.append(gold_triple["emotion"])
        pred_emotions.append(pred_triple["emotion"])

# -----------------------------
# SAVE MISMATCHED ROWS
# -----------------------------
if mismatches:
    mismatch_path = f"emotion_only_mismatches_{input_name}.jsonl"
    with open(mismatch_path, "w", encoding="utf-8") as f:
        for row in mismatches:
            f.write(json.dumps(row, ensure_ascii=False) + "\n")

    print(f"\n⚠️  Found {len(mismatches)} mismatched rows")
    print(f"Saved mismatches → {mismatch_path}")
else:
    print("\n✅ No triple-count mismatches found")

print(f"Used {len(gold_emotions)} aligned emotion labels for evaluation")

# -----------------------------
# Compute Metrics (Emotion Only)
# -----------------------------
emotion_macro = f1_score(
    gold_emotions, pred_emotions,
    average="macro", zero_division=0
)

emotion_micro = f1_score(
    gold_emotions, pred_emotions,
    average="micro", zero_division=0
)

correct = sum(1 for g, p in zip(gold_emotions, pred_emotions) if g == p)
emotion_accuracy = correct / len(gold_emotions) if gold_emotions else 0.0

# -----------------------------
# PRINT REPORT
# -----------------------------
print("\n==============================")
print("EMOTION-ONLY EVALUATION REPORT")
print("==============================\n")

print("Emotion Macro-F1      :", round(emotion_macro, 4))
print("Emotion Micro-F1      :", round(emotion_micro, 4))
print("Exact Emotion Accuracy:", round(emotion_accuracy, 4))

print("\n==============================")

print("DETAILED EMOTION REPORT")
print("\n==============================")


detailed_classification = classification_report(
    gold_emotions,
    pred_emotions,
    zero_division=0
)

header = "emotion           precision    recall  f1-score   support"
detailed_classification = header + "\n" + detailed_classification

print(detailed_classification)

# -----------------------------
# SAVE METRICS CSV
# -----------------------------
df = pd.DataFrame({
    "Emotion Macro F1": [emotion_macro],
    "Emotion Micro F1": [emotion_micro],
    "Emotion Accuracy": [emotion_accuracy],
    "Detailed Report": [detailed_classification]
})

out_csv = f"f1_metrics_{input_name}.csv"
df.to_csv(out_csv, index=False)

print(f"\nSaved → {out_csv}")

Loaded gold rows: 299
Loaded pred rows: 299

✅ No triple-count mismatches found
Used 583 aligned emotion labels for evaluation

EMOTION-ONLY EVALUATION REPORT

Emotion Macro-F1      : 0.4141
Emotion Micro-F1      : 0.4957
Exact Emotion Accuracy: 0.4957

DETAILED EMOTION REPORT

emotion           precision    recall  f1-score   support
                precision    recall  f1-score   support

    admiration       0.48      0.59      0.53        74
     annoyance       0.50      0.74      0.60       119
disappointment       0.44      0.33      0.38       104
       disgust       0.50      0.29      0.36         7
     gratitude       0.43      0.33      0.38         9
mentioned_only       0.61      0.59      0.60        90
mixed_emotions       0.20      0.08      0.11        13
    no_emotion       0.32      0.21      0.25        67
  satisfaction       0.54      0.50      0.52       100

      accuracy                           0.50       583
     macro avg       0.45      0.41      0.41

In [3]:
# ## Aspect-Polarity-Emotion evaluation

# import json
# import os
# import pandas as pd
# from sklearn.metrics import f1_score, classification_report
# from collections import Counter

# # -----------------------------
# # Paths
# # -----------------------------

# #GOLD_PATH = os.path.join(data_root, "daniel_50.jsonl")
# #PRED_PATH = os.path.join(results_root, "gemini-flash", "gemini_annotated_aspect_polarity_emotions_daniel_50.jsonl")

# GOLD_PATH = os.path.join(data_root, "cleaned_300.jsonl")
# PRED_PATH = os.path.join(results_root, "gemini-flash", "gemini_annotated_aspect_polarity_emotions_300.jsonl")
# # -----------------------------
# # Load JSONL
# # -----------------------------
# def load_jsonl(path):
#     return [json.loads(line) for line in open(path, "r", encoding="utf-8")]

# gold = load_jsonl(GOLD_PATH)
# pred = load_jsonl(PRED_PATH)

# # Map gold/pred by input text
# gold_map = {row["input"]: row["output"] for row in gold}
# pred_map = {row["input"]: row["output"] for row in pred}

# print(f"Loaded gold rows: {len(gold_map)}")
# print(f"Loaded pred rows: {len(pred_map)}")


# # -----------------------------------
# # Duplicate detection helper
# # -----------------------------------
# def find_duplicates(rows, name):
#     counter = Counter([row["input"] for row in rows])
#     dups = [inp for inp, c in counter.items() if c > 1]

#     print(f"\n==============================")
#     print(f"DUPLICATE CHECK → {name}")
#     print("==============================")

#     if not dups:
#         print("No duplicates found.\n")
#         return {}

#     print(f"Found {len(dups)} duplicate input texts:")
#     for d in dups:
#         print(f" - {d!r}  (x{counter[d]})")
#     print()

#     # return mapping of input → list of rows
#     dup_map = {d: [r for r in rows if r["input"] == d] for d in dups}
#     return dup_map


# from collections import defaultdict

# def find_duplicate_indices_1_based(rows, name):
#     index_map = defaultdict(list)

#     for i, row in enumerate(rows):
#         index_map[row["input"]].append(i + 1)   # convert to 1-based

#     duplicates = {text: idxs for text, idxs in index_map.items() if len(idxs) > 1}

#     print(f"\n==============================")
#     print(f"DUPLICATE ROW INDICES → {name}")
#     print("==============================")

#     if not duplicates:
#         print("No duplicates found.\n")
#         return duplicates

#     for text, idxs in duplicates.items():
#         print(f"\nTEXT: {text!r}")
#         print(f"1-based row numbers: {idxs}")

#     print()
#     return duplicates


# gold_dup_indices = find_duplicate_indices_1_based(gold, "GOLD FILE")
# pred_dup_indices = find_duplicate_indices_1_based(pred, "PRED FILE")

# # -----------------------------------
# # Duplicates appearing in BOTH (with indices)
# # -----------------------------------
# print("\n==============================")
# print("DUPLICATES IN BOTH FILES (WITH ROW NUMBERS)")
# print("==============================")

# common = set(gold_dup_indices.keys()) & set(pred_dup_indices.keys())

# if not common:
#     print("No overlapping duplicates.\n")
# else:
#     for text in common:
#         print(f"\nTEXT: {text!r}")
#         print(f" → GOLD rows: {gold_dup_indices[text]}")
#         print(f" → PRED rows: {pred_dup_indices[text]}")
#         print("\n-----------------------------------")

# # -----------------------------
# # Triple alignment
# # -----------------------------
# def align_triples(gold_list, pred_list):
#     """
#     Ensures aligned triples for scoring:
#     - If pred has MORE triples → truncate
#     - If pred has FEWER triples → pad with ('none','none','none')
#     """
#     g = [(t["aspect"], t["polarity"], t["emotion"]) for t in gold_list]
#     p = [(t["aspect"], t["polarity"], t["emotion"]) for t in pred_list]

#     gold_n = len(g)
#     pred_n = len(p)

#     # truncate hallucinations
#     if pred_n > gold_n:
#         p = p[:gold_n]

#     # pad missing predictions
#     if pred_n < gold_n:
#         pad = [("none", "none", "none")] * (gold_n - pred_n)
#         p.extend(pad)

#     return g, p


# # -----------------------------
# # Collect aligned labels
# # -----------------------------
# all_gold_as, all_pred_as = [], []
# all_gold_pol, all_pred_pol = [], []
# all_gold_emo, all_pred_emo = [], []

# gold_triples_full = []
# pred_triples_full = []

# for text, gold_list in gold_map.items():

#     pred_list = pred_map.get(text, [])

#     g_aligned, p_aligned = align_triples(gold_list, pred_list)

#     for (ga, gp, ge), (pa, pp, pe) in zip(g_aligned, p_aligned):
#         all_gold_as.append(ga)
#         all_gold_pol.append(gp)
#         all_gold_emo.append(ge)

#         all_pred_as.append(pa)
#         all_pred_pol.append(pp)
#         all_pred_emo.append(pe)

#         gold_triples_full.append((ga, gp, ge))
#         pred_triples_full.append((pa, pp, pe))


# # -----------------------------
# # Compute metrics
# # -----------------------------
# aspect_f1 = f1_score(all_gold_as, all_pred_as, average="macro", zero_division=0)
# polarity_f1 = f1_score(all_gold_pol, all_pred_pol, average="macro", zero_division=0)
# emotion_macro = f1_score(all_gold_emo, all_pred_emo, average="macro", zero_division=0)
# emotion_micro = f1_score(all_gold_emo, all_pred_emo, average="micro", zero_division=0)

# # Full ABSA accuracy (exact triple match)
# match = sum(1 for g, p in zip(gold_triples_full, pred_triples_full) if g == p)
# exact_acc = match / len(gold_triples_full)


# # -----------------------------
# # Pretty PRINT
# # -----------------------------
# print("\n==============================")
# print("FINAL EVALUATION REPORT")
# print("==============================\n")

# print("ASPECT CLASSIFICATION")
# print("------------------------------")
# print(f"Macro F1: {aspect_f1:.4f}\n")

# print("POLARITY CLASSIFICATION")
# print("------------------------------")
# print(f"Macro F1: {polarity_f1:.4f}\n")

# print("EMOTION CLASSIFICATION")
# print("------------------------------")
# print(f"Emotion Macro-F1 : {emotion_macro:.4f}")
# print(f"Emotion Micro-F1 : {emotion_micro:.4f}\n")

# print("FULL ABSA TRIPLE MATCH")
# print("------------------------------")
# print(f"Full ABSA Accuracy: {exact_acc:.4f}\n")

# print("==============================")
# print("DETAILED CLASSIFICATION REPORT")
# print("==============================\n")
# print(classification_report(all_gold_emo, all_pred_emo, zero_division=0))


# # Save optional CSV
# df = pd.DataFrame({
#     "Aspect F1":      [aspect_f1],
#     "Polarity F1":    [polarity_f1],
#     "Emotion F1":     [emotion_macro],
#     "Emotion Micro":  [emotion_micro],
#     "Full ABSA Acc":  [exact_acc]
# })
# df.to_csv(f"eval_results_absa_{GOLD_PATH.split('/')[-1].replace('.jsonl', '')}.csv", index=False)

# print(f"\nSaved → eval_results_{GOLD_PATH.split('/')[-1].replace('.jsonl', '')}.csv")